In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
# 让画图内置
%matplotlib inline

# 忽略版本号之类的warnings
import warnings
warnings.filterwarnings('ignore')
import os
from datetime import date

# 数据导入

In [3]:
# 导入数据
# original_data = pd.read_csv('data\processed_data.csv')
# data = original_data.copy()
# data.shape

In [4]:
# 导入数据
data = pd.read_csv('data\processed_data.csv')
data.shape

(42413557, 11)

In [5]:
data['event_time'] = pd.to_datetime(data['event_time'])
data['category_code'] = data['category_code'].astype('category')
data['brand'] = data['brand'].astype('category')

data['event_day'] = data['event_time'].dt.date
data['event_hour'] = data['event_time'].dt.hour

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42413557 entries, 0 to 42413556
Data columns (total 13 columns):
 #   Column         Dtype         
---  ------         -----         
 0   event_time     datetime64[ns]
 1   event_type     object        
 2   product_id     int64         
 3   category_id    int64         
 4   category_code  category      
 5   brand          category      
 6   price          float64       
 7   user_id        int64         
 8   category       object        
 9   sub_category   object        
 10  product_name   object        
 11  event_day      object        
 12  event_hour     int32         
dtypes: category(2), datetime64[ns](1), float64(1), int32(1), int64(3), object(5)
memory usage: 3.4+ GB


In [7]:
data.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,category,sub_category,product_name,event_day,event_hour
0,2019-10-01 00:00:00,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,NaN,NaN,NaN,2019-10-01,0
1,2019-10-01 00:00:00,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,appliances,environment,water_heater,2019-10-01,0
2,2019-10-01 00:00:01,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,furniture,living_room,sofa,2019-10-01,0
3,2019-10-01 00:00:01,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,computers,notebook,notebook,2019-10-01,0
4,2019-10-01 00:00:04,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,electronics,smartphone,smartphone,2019-10-01,0


# 热销商品
- '下单数','动销商品数','gmv','单均价'

In [8]:
# 商品价值
product_info = data.query('event_type == "purchase"').groupby('event_type').agg({'product_id': ['count','nunique'],'price': ['sum','mean']})
product_info.columns = ['下单数','动销商品数','gmv','单均价']
product_info

,下单数,动销商品数,gmv,单均价
event_type,,,,
purchase,742773,42241,2.299332e+08,309.560542


## 分类别

In [15]:
product_info_bycategory = data.query('event_type == "purchase"').groupby('category').agg({'product_id': ['count','nunique'],'price': ['sum','mean']})
product_info_bycategory.columns = ['下单数','动销商品数','gmv','单均价']
product_info_bycategory.sort_values(by='gmv',ascending=False)

,下单数,动销商品数,gmv,单均价
category,,,,
electronics,422979,4698,1.764456e+08,417.149735
appliances,74988,4838,1.358176e+07,181.119120
computers,27853,2638,1.137787e+07,408.497139
furniture,8299,1634,1.673243e+06,201.619835
auto,10619,609,1.273956e+06,119.969461
construction,7801,939,9.329950e+05,119.599413
kids,5482,1007,6.781407e+05,123.703150
apparel,8002,2512,6.249378e+05,78.097694
sport,1236,270,3.225590e+05,260.970065


## 分小类

In [19]:
product_info_bysubcate = data.query('event_type == "purchase"').groupby(['category','sub_category']).agg({'product_id': ['count','nunique'],'price': ['sum','mean']})

product_info_bysubcate.columns =  ['下单数','动销商品数','gmv','单均价']

product_info_bysubcate.query('下单数 > 0').sort_values('gmv',ascending=False)[:10]

下单数  动销商品数           gmv         单均价
category    sub_category                                         
electronics smartphone    337979    924  1.570339e+08  464.626246
appliances  kitchen        48119   3084  1.053302e+07  218.895239
computers   notebook       15588    614  8.978883e+06  576.012537
electronics video          21643    406  8.459179e+06  390.850590
            clocks         17903   1439  4.817089e+06  269.066025
            audio          35590   1356  4.151286e+06  116.641928
appliances  environment    18073    953  2.320239e+06  128.381508
electronics tablet          5602    222  1.610917e+06  287.561123
auto        accessories    10619    609  1.273956e+06  119.969461
computers   desktop         3232    445  1.116822e+06  345.551355

## 分商品名 

In [17]:
product_info_bypname = data.query('event_type == "purchase"').groupby('product_name').agg({'product_id': ['count','nunique'],'price': ['sum','mean']})

product_info_bypname.columns = ['下单数','动销商品数','gmv','单均价']

product_info_bypname.query('下单数 > 0').sort_values('gmv',ascending=False)[:20]

,下单数,动销商品数,gmv,单均价
product_name,,,,
smartphone,337979,924,1.570339e+08,464.626246
notebook,15588,614,8.978883e+06,576.012537
tv,21561,387,8.422119e+06,390.618217
clocks,17903,1439,4.817089e+06,269.066025
washer,16146,342,4.658223e+06,288.506346
refrigerators,11218,624,3.830077e+06,341.422447
headphone,30501,678,3.538807e+06,116.022661
vacuum,12378,414,1.716425e+06,138.667427
tablet,5602,222,1.610917e+06,287.561123


## 分品牌

In [18]:
product_info_bybrand = data.query('event_type == "purchase"').groupby('brand').agg({'product_id': ['count','nunique'],'price': ['sum','mean']})

product_info_bybrand.columns = ['下单数','动销商品数','gmv','单均价']

product_info_bybrand.query('下单数 > 0').sort_values('gmv',ascending=False)[:20]

,下单数,动销商品数,gmv,单均价
brand,,,,
apple,142858,394,1.111995e+08,778.391800
samsung,172878,687,4.640204e+07,268.409152
xiaomi,56609,474,9.192640e+06,162.388317
huawei,23499,104,4.883105e+06,207.800533
acer,6880,174,3.575716e+06,519.726118
lg,8725,257,3.387361e+06,388.236207
lucente,11576,445,3.123439e+06,269.820228
sony,6729,442,2.478197e+06,368.286028
oppo,10887,35,2.412034e+06,221.551757


## 分商品id

In [13]:
product_info_bybrand = data.query('event_type == "purchase"').groupby('product_id').agg({'product_id': ['count','nunique'],'price': ['sum','mean']})

product_info_bybrand.columns = ['下单数','动销商品数','gmv','单均价']

product_info_bybrand.query('下单数 > 0')

,下单数,动销商品数,gmv,单均价
product_id,,,,
1000978,12,1,3840.13,320.010833
1001588,12,1,1538.18,128.181667
1002042,3,1,231.42,77.140000
1002062,18,1,1723.47,95.748333
1002098,13,1,4818.32,370.640000
...,...,...,...,...
59000007,1,1,47.88,47.880000
60400006,1,1,332.05,332.050000
60500001,4,1,102.34,25.585000


## 分下单方式
- 直接下单 / 加购后下单